In [65]:
import cv2
import mmcv
import numpy as np
import os
import sys
from multiprocessing import Pool
from os import path as osp
import glob as gl
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.merge import merge
import glob as gl
from osgeo import gdal,ogr,osr,gdalconst
from affine import Affine
from skimage import exposure
from skimage.util import img_as_ubyte
import fiona
import rasterio
import rasterio.mask
from rasterio.windows import Window
from rasterio.features import bounds
from rasterio.transform import rowcol
import math
import geopandas as gpd
from shapely.geometry import mapping
from shapely.geometry import box
from raster_util import *

Planet - green, red, nir
<br>
Corresponds to bands 3,4,5 of landsat (g,r,nir)

First, save Landsat as a single image with band order green, red, nir:

In [ ]:
im_folder = r'D:\Data\Landsat_Imagery\landsat_example_landsat8l1g_analytic\files\Landsat8L1G\LC80460162020236LGN00\analytic'

In [ ]:
stack_tif(im_folder, 'landsat_subset_test/landsat.tif', band_order = [3,4,5])

Then, chunk the landsat into tiles of size 48, with overlap:

In [ ]:
chunk_tif('landsat_subset_test/landsat.tif', 48, 'D:/Data/Landsat_Imagery/for_esrgan/subsets/tifs/')

Rescale the reflectance of these images and convert them to pngs (this function does it all at once...):

In [ ]:
save_as_pngs('D:/Data/Landsat_Imagery/for_esrgan/subsets/tifs/', 'D:/Data/Landsat_Imagery/for_esrgan/subsets/pngs_5p_stretch/')

Now you run these chunked pngs through ESRGAN!

---

Once you have output pngs from ESRGAN, you must turn them into tifs with associated spatial information:

In [ ]:
make_tifs_pngs(r'C:\Users\elezine\OneDrive\Academics\Research\Brown\code\github reps\super-resolution\pixel-smasher\results\Landsat_Test\visualization\landsat_test', r'D:\Data\Landsat_Imagery\for_esrgan\subsets\tifs', r'D:\Data\Landsat_Imagery\for_esrgan\subsets\sr_tifs_5p_stretch')

After doing that, you will want to crop the super resolved images to fix edge effects...

In [ ]:
crop_tifs(r'D:\Data\Landsat_Imagery\for_esrgan\subsets\sr_tifs_5p_stretch\\', 120, 0)

And then merge together some of these tifs that intersect with some bounding box (defined by a shpfile):

In [63]:
folder_path = r'D:\Data\Landsat_Imagery\for_esrgan\subsets\sr_tifs_5p_stretch\\'
shpfile_path = r'D:\Data\Planet_Imagery\comp_psscene4band_analytic_sr_udm2\files\PSScene4Band\20200823_184406_0f34\analytic_sr_udm2\extent_shpfile.shp'
shape_file = gpd.read_file(shpfile_path)

This finds any files that intersect with a bounding box:

In [ ]:
intersections = []

tifs = gl.glob(folder_path + '/*.tif')
tif_list_inter = []
tif_list = []
for tif in tifs[7000:15000]:
    src = rio.open(tif)
    shape_file_reproj = shape_file.to_crs(src.crs)

    bounds = src.bounds
    geom = box(*bounds)
    
    df = gpd.GeoDataFrame({"id":1,"geometry":[geom]}, crs= 'epsg:32612')
    
    if df.intersects(shape_file_reproj).values[0]:
        print('intersection')
        intersections.append(tif)

This merges those images:

In [53]:
tif_list = []
for tif in intersections:
    tif_list.append(rio.open(tif))

img, transform = merge(tif_list)

This saves those merged images:

In [62]:
profile = tif_list[0].profile
profile.update({'transform':transform})
profile.update({'width':img.shape[2], 'height':img.shape[1]})
profile.update({'dtype':'uint8'})

with rio.open(r'D:\Data\Landsat_Imagery\for_esrgan\merged.tif', "w", **profile) as dest:
    dest.write(img)